In [ ]:
%pip install -qU langchain-mistralai


In [ ]:
!pip install langchain
!pip install faiss-cpu
!pip install openai
!pip install -U langchain-community
!pip install unstructured
!pip install langchain_openai

In [ ]:
!pip install tiktoken

In [ ]:
import getpass
import os

os.environ["MISTRALAI_API_KEY"] = getpass.getpass("Enter your MistralAI API key: ")

Enter your MistralAI API key: ··········


In [ ]:

from langchain_mistralai import MistralAIEmbeddings

embeddings = MistralAIEmbeddings(model="mistral-embed", mistral_api_key=os.environ["MISTRALAI_API_KEY"])

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
# from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader('/content/Bayesian_Belief_Network,Exact_inference,approx_inference,_causal_network[1].pdf')
documents = loader.load()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                      chunk_overlap=200)
docs = text_splitter.split_documents(documents)

In [ ]:
docs


[Document(metadata={'source': '/content/Bayesian_Belief_Network,Exact_inference,approx_inference,_causal_network[1].pdf', 'page': 0}, page_content='1.Bayesian Belief Network in artificial intelligence  \nBayesian belief network is key computer technology for dealing with probabilistic events \nand to solve a problem which has uncertainty. We can define a Bayesian network as:  \n"A Bayesian network is a probab ilistic graphical model which represents a set of variables \nand their conditional dependencies using a directed acyclic graph."  \nIt is also called a  Bayes network, belief network, decision network , or Bayesian model . \nBayesian networks are probabilistic, bec ause these networks are built from a  probability \ndistribution , and also use probability theory for prediction and anomaly detection.  \nReal world applications are probabilistic in nature, and to represent the relationship \nbetween multiple events, we need a B ayesian network. It can also be used in various tasks 

In [ ]:
from langchain_community.vectorstores import FAISS

faiss_index = FAISS.from_documents(docs, embeddings)

In [ ]:
retriever = faiss_index.as_retriever()

In [ ]:

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:


{context}


Question: {question}""")

In [ ]:
from langchain_mistralai.chat_models import ChatMistralAI
model = ChatMistralAI(mistral_api_key=os.getenv("MISTRALAI_API_KEY"))

In [ ]:

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
user_question = input("Kindly provide the details you would like to summit : ")

# Invoke the RAG chain with the user's question
result = rag_chain.invoke(user_question)
print(result)

Kindly provide the details you would like to summit : What are the main components of Bayesian network?
The main components of a Bayesian network are:

1. Causal Component: This refers to the structure of the Bayesian network, which is a directed acyclic graph (DAG) representing the causal relationships or conditional dependencies among the variables.
2. Actual numbers: This refers to the numerical values in the Bayesian network, which are the conditional probability distributions (CPDs) associated with each node in the network. The CPDs specify the probability of each value of a node, given the values of its parent nodes.
